In [1]:
from numpy import average
import os
import graphlab as gl
import numpy as np
import gensim
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

gl.canvas.set_target('ipynb')

[INFO] This non-commercial license of GraphLab Create is assigned to marvin.bertin@gmail.comand will expire on September 15, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-3502 - Server binary: /Users/marvinbertin/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1444103072.log
[INFO] GraphLab Server Version: 1.6.1


In [2]:
traindata_path = "/Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/labeledTrainData.tsv"

In [3]:
train_data = gl.SFrame.read_csv(traindata_path,header=True, delimiter='\t',quote_char='"',
                                column_type_hints = {'id':str, 'sentiment' : int, 'review':str } )

PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/labeledTrainData.tsv
PROGRESS: Parsing completed. Parsed 100 lines in 0.563292 secs.
PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/labeledTrainData.tsv
PROGRESS: Parsing completed. Parsed 25000 lines in 0.900316 secs.


In [ ]:
unlabeledtraindata_path = "/Users/marvinbertin/graphlab_data/Bags-of-Popcorn/UnlabeledData/unlabeledTrainData.tsv"
unlabeled_train_data = gl.SFrame.read_csv(traindata_path,header=True, delimiter='\t',quote_char='"',
                                column_type_hints = {'id':str, 'review':str } )

In [4]:
train_data.show()

In [5]:
train_data.head()

Columns:
	id	str
	sentiment	int
	review	str

Rows: 10

Data:
+---------+-----------+-------------------------------+
|    id   | sentiment |             review            |
+---------+-----------+-------------------------------+
|  5814_8 |     1     | With all this stuff going ... |
|  2381_9 |     1     | "The Classic War of the Wo... |
|  7759_3 |     0     | The film starts with a man... |
|  3630_4 |     0     | It must be assumed that th... |
|  9495_8 |     1     | Superbly trashy and wondro... |
|  8196_8 |     1     | I dont know why people thi... |
|  7166_2 |     0     | This movie could have been... |
| 10633_1 |     0     | I watched this video at a ... |
|  319_1  |     0     | A friend of mine bought th... |
| 8713_10 |     1     | <br /><br />This movie is ... |
+---------+-----------+-------------------------------+
[10 rows x 3 columns]

In [6]:
train_data['review'].head(1)

dtype: str
Rows: 1
["With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit

In [8]:
BASE_DIR = "/Users/marvinbertin/graphlab_data/Bags-of-Popcorn"
os.mkdir("%s/txt" % BASE_DIR)
train_data.apply(lambda r: file("%s/txt/%s.txt" % (BASE_DIR, r["id"]),"w").write(r['review'])).__materialize__()

In [ ]:
unlabeled_train_data.apply(lambda r: file("%s/txt/%s.txt" % (BASE_DIR, r["id"]),"w").write(r['review'])).__materialize__()

In [9]:
class TrainSentences(object):
    """
    Iterator class that returns Sentences from texts files in a input directory
    """
    RE_WIHTE_SPACES = re.compile("\s+")
    STOP_WORDS = set(stopwords.words("english"))
    def __init__(self, dirname):
        """
        Initialize a TrainSentences object with a input directory that contains text files for training
        :param dirname: directory name which contains the text files        
        """
        self.dirname = dirname

    def __iter__(self):
        """
        Sentences iterator that return sentences parsed from files in the input directory.
        Each sentences is returned as list of words
        """
        #First iterate  on all files in the input directory
        for fname in os.listdir(self.dirname):
            # read line from file (Without reading the entire file)
            for line in file(os.path.join(self.dirname, fname), "rb"):
                # split the read line into sentences using NLTK
                for s in txt2sentences(line, is_html=True):
                    # split the sentence into words using regex
                    w =txt2words(s, lower=True, is_html=False, remove_stop_words=False,
                                                 remove_none_english_chars=True)
                    #skip short sentences with less than 3 words
                    if len(w) < 3:
                        continue
                    yield w

def txt2sentences(txt, is_html=False, remove_none_english_chars=True):
    """
    Split the English text into sentences using NLTK
    :param txt: input text.
    :param is_html: If True thenremove HTML tags using BeautifulSoup
    :param remove_none_english_chars: if True then remove non-english chars from text
    :return: string in which each line consists of single sentence from the original input text.
    :rtype: str
    """
    if is_html:
        txt = BeautifulSoup(txt).get_text()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

    # split text into sentences using nltk packages
    for s in tokenizer.tokenize(txt):
        if remove_none_english_chars:
            #remove none English chars
            s = re.sub("[^a-zA-Z]", " ", s)
        yield s

In [11]:
def txt2words(txt, lower=True, is_html=False, remove_none_english_chars=True, remove_stop_words=True):
    """
    Split text into words list
    :param txt: the input text
    :param lower: if to make the  text to lowercase or not.
    :param is_html: If True then  remove HTML tags using BeautifulSoup
    :param remove_none_english_chars: if True then remove non-english chars from text
    :param remove_stop_words: if True then remove stop words from text
    :return: words list create from the input text according to the input parameters.
    :rtype: list
    """
    if is_html:
        txt = BeautifulSoup(txt).get_text()
    if lower:
        txt = txt.lower()
    if remove_none_english_chars:
        txt = re.sub("[^a-zA-Z]", " ", txt)

    words = TrainSentences.RE_WIHTE_SPACES.split(txt.strip().lower())
    if remove_stop_words:
        #remove stop words from text
        words = [w for w in words if w not in TrainSentences.STOP_WORDS]
    return words

In [12]:
sentences = TrainSentences("%s/LabeledData" % BASE_DIR)
model = gensim.models.Word2Vec(sentences, size=300, workers=4, min_count=40)
model.save("%s/movie_reviews_300_c_40.word2vec" % BASE_DIR)

In [20]:
model.most_similar("horrible")

[(u'terrible', 0.7815021276473999),
 (u'horrendous', 0.709304690361023),
 (u'dreadful', 0.6580719947814941),
 (u'lousy', 0.623784065246582),
 (u'awful', 0.620533287525177),
 (u'horrid', 0.6181655526161194),
 (u'atrocious', 0.5620525479316711),
 (u'abysmal', 0.5584285259246826),
 (u'pathetic', 0.5454639196395874),
 (u'laughable', 0.5300486087799072)]

In [19]:
model.most_similar("terrific")

[(u'fantastic', 0.7257372140884399),
 (u'superb', 0.7204450368881226),
 (u'splendid', 0.7123845219612122),
 (u'marvelous', 0.6995391845703125),
 (u'wonderful', 0.6877990365028381),
 (u'fine', 0.6634610295295715),
 (u'marvellous', 0.6486601829528809),
 (u'fabulous', 0.6358596086502075),
 (u'superlative', 0.6269110441207886),
 (u'outstanding', 0.6199491620063782)]

In [21]:
train_data.num_rows

<bound method SFrame.num_rows of Columns:
	id	str
	sentiment	int
	review	str

Rows: 25000

Data:
+---------+-----------+-------------------------------+
|    id   | sentiment |             review            |
+---------+-----------+-------------------------------+
|  5814_8 |     1     | With all this stuff going ... |
|  2381_9 |     1     | "The Classic War of the Wo... |
|  7759_3 |     0     | The film starts with a man... |
|  3630_4 |     0     | It must be assumed that th... |
|  9495_8 |     1     | Superbly trashy and wondro... |
|  8196_8 |     1     | I dont know why people thi... |
|  7166_2 |     0     | This movie could have been... |
| 10633_1 |     0     | I watched this video at a ... |
|  319_1  |     0     | A friend of mine bought th... |
| 8713_10 |     1     | <br /><br />This movie is ... |
+---------+-----------+-------------------------------+
[25000 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n

In [22]:
class DeepTextAnalyzer(object):
    def __init__(self, word2vec_model):
        """
        Construct a DeepTextAnalyzer using the input Word2Vec model
        :param word2vec_model: a trained Word2Vec model
        """
        self._model = word2vec_model

    def txt2vectors(self,txt, is_html):
        """
        Convert input text into an iterator that returns the corresponding vector representation of each
        word in the text, if it exists in the Word2Vec model
        :param txt: input text
        :param is_html: if True, then extract the text from the input HTML
        :return: iterator of vectors created from the words in the text using the Word2Vec model.
        """
        words = txt2words(txt,is_html=is_html, lower=True, remove_none_english_chars=True)
        words = [w for w in words if w in self._model]
        if len(words) != 0:
            for w in words:
                yield self._model[w]


    def txt2avg_vector(self, txt, is_html):
        """
        Calculate the average vector representation of the input text
        :param txt: input text
        :param is_html: is the text is a HTML
        :return the average vector of the vector representations of the words in the text  
        """
        vectors = self.txt2vectors(txt,is_html=is_html)
        vectors_sum = next(vectors, None)
        if vectors_sum is None:
            return None
        count =1.0
        for v in vectors:
            count += 1
            vectors_sum = np.add(vectors_sum,v)
        
        #calculate the average vector and replace +infy and -inf with numeric values 
        avg_vector = np.nan_to_num(vectors_sum/count)
        return avg_vector

In [23]:
dt = DeepTextAnalyzer(model)
train_data['vectors'] = train_data['review'].apply(lambda p: dt.txt2avg_vector(p, is_html=True))
train_data['vectors'].head(1)

dtype: array
Rows: 1
[array('d', [0.03502282872796059, 0.01633261889219284, -0.01777825504541397, 0.006104846950620413, 0.014137832447886467, -0.018585577607154846, 0.015146004036068916, -0.02158847078680992, -0.006849241908639669, 0.00038025836693122983, -0.009065904654562473, 0.0200472641736269, 0.011096293106675148, -0.004228357225656509, -0.001332852290943265, 0.03325987234711647, 0.02252323366701603, 0.011898208409547806, -0.027523566037416458, 0.024843977764248848, -0.06615491211414337, 0.01505744643509388, 0.05482500046491623, 0.0079379016533494, -0.03465285897254944, -0.00264940457418561, 0.024672172963619232, -0.015375135466456413, -0.024094343185424805, -0.026456154882907867, -0.03384796157479286, 0.037299130111932755, -0.012322157621383667, 0.017702100798487663, -0.03666450083255768, -0.009129147976636887, 0.01178585086017847, 0.01688515953719616, -0.001407705363817513, -0.03739621862769127, -0.01011306419968605, -0.0057843648828566074, 0.023623857647180557, -0.0399926938116

In [24]:
train_data.num_rows()

25000

In [27]:
train_data.head()

Columns:
	id	str
	sentiment	int
	review	str
	vectors	array

Rows: 10

Data:
+---------+-----------+-------------------------------+
|    id   | sentiment |             review            |
+---------+-----------+-------------------------------+
|  5814_8 |     1     | With all this stuff going ... |
|  2381_9 |     1     | "The Classic War of the Wo... |
|  7759_3 |     0     | The film starts with a man... |
|  3630_4 |     0     | It must be assumed that th... |
|  9495_8 |     1     | Superbly trashy and wondro... |
|  8196_8 |     1     | I dont know why people thi... |
|  7166_2 |     0     | This movie could have been... |
| 10633_1 |     0     | I watched this video at a ... |
|  319_1  |     0     | A friend of mine bought th... |
| 8713_10 |     1     | <br /><br />This movie is ... |
+---------+-----------+-------------------------------+
+-------------------------------+
|            vectors            |
+-------------------------------+
| [0.035022828728, 0.0163326... |
| [-0.0267988946289, 0.00687... |
| [0.0674279406667, 0.006555... |
| [0.0392956994474, 0.038071... |
| [0.0449821017683, 0.011471... |
| [0.0180358290672, 0.022768... |
| [-0.0313609987497, 0.02456... |
| [0.0339717939496, 0.048592... |
| [0.00447202892974, 0.01647... |
| [0.032674357295, 0.0182836... |
+-------------------------------+
[10 rows x 4 columns]

In [26]:
train_subset, test_subset = train_data.random_split(0.8, seed=1)

In [28]:
cls = gl.classifier.create(train_subset, target="sentiment", features=["vectors"])
result1 = cls.evaluate(test_subset)
result1

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 18930
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 300
PROGRESS: Number of coefficients    : 301
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+---

{'accuracy': 0.8534072900158478, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  392  |
 |      1       |        0        |  348  |
 |      1       |        1        |  2216 |
 |      0       |        0        |  2092 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [31]:
cls = gl.classifier.neuralnet_classifier.create(train_subset, target="sentiment", features=["vectors"],
                                                metric=['accuracy', 'recall@2'],
                                                max_iterations=3)
result2 = cls.evaluate(test_subset)
result2

Using network:

### network layers ###
layer[0]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[1]: SigmoidLayer
layer[2]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 2
layer[3]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Creating neuralnet using cpu
PROGRESS: Training with batch size = 100
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+
PROGRESS: | Iteration | Examples | Elapsed Time | Training-accuracy | Validation-accuracy | Training-recall@2 | Validation-recall@2 

{'accuracy': 0.4920760691165924, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 2
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  2484 |
 |      1       |        0        |  2564 |
 +--------------+-----------------+-------+
 [2 rows x 3 columns]}

In [32]:
cls = gl.classifier.random_forest_classifier.create(train_subset, target="sentiment", features=["vectors"])
result3 = cls.evaluate(test_subset)
result3

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Random forest classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 18879
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 300
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   7.592e-01   7.018e-01        1.10s
PROGRESS:      1   7.815e-01   7.409e-01        1.82s
PROGRESS:      2   7.943e-01   7.418e-01        2.54s
PROGRESS:      3   7.959e-01   7.418e-01        3.27s
PROGRESS:      4   7.986e-01   7.521e-01        4.03s
PROGRESS:      5   8.006e-01   7.577e-01        4.77s
PROGRESS:      6   8.058e-01   7.

{'accuracy': 0.7670364500792393, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  600  |
 |      1       |        1        |  1988 |
 |      1       |        0        |  576  |
 |      0       |        0        |  1884 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [33]:
cls = gl.classifier.boosted_trees_classifier.create(train_subset, target="sentiment", features=["vectors"])
result4 = cls.evaluate(test_subset)
result4

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 19002
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 300
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   7.668e-01   7.358e-01        1.17s
PROGRESS:      1   7.920e-01   7.432e-01        2.35s
PROGRESS:      2   8.158e-01   7.800e-01        3.54s
PROGRESS:      3   8.250e-01   7.832e-01        4.82s
PROGRESS:      4   8.365e-01   7.937e-01        6.17s
PROGRESS:      5   8.469e-01   7.853e-01        7.45s
PROGRESS:      6   8.530e-01   7.

{'accuracy': 0.8019017432646592, 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        1        |  515  |
 |      1       |        1        |  2079 |
 |      1       |        0        |  485  |
 |      0       |        0        |  1969 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns]}

In [34]:
cls = gl.classifier.create(train_data, target='sentiment', features=['vectors'])

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: LogisticClassifier, SVMClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.
PROGRESS: Logistic regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 23852
PROGRESS: Number of classes           : 2
PROGRESS: Number of feature columns   : 1
PROGRESS: Number of unpacked features : 300
PROGRESS: Number of coefficients    : 301
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+-------------------+---------------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-accuracy | Validation-accuracy |
PROGRESS: +-----------+----------+---

In [35]:
#creating the test dataset
testdata_path = "/Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/testData.tsv"
test_data = gl.SFrame.read_csv(testdata_path,header=True, delimiter='\t',quote_char='"',
                               column_type_hints = {'id':str, 'review':str } )

PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/testData.tsv
PROGRESS: Parsing completed. Parsed 100 lines in 0.5154 secs.
PROGRESS: Finished parsing file /Users/marvinbertin/graphlab_data/Bags-of-Popcorn/LabeledData/testData.tsv
PROGRESS: Parsing completed. Parsed 25000 lines in 0.906346 secs.


In [36]:
dt = DeepTextAnalyzer(model)
test_data['vectors'] = test_data['review'].apply(lambda p: dt.txt2avg_vector(p, is_html=True))
test_data['vectors'].head(1)

dtype: array
Rows: 1
[array('d', [-0.005660160444676876, -0.030306031927466393, -0.07506293058395386, 0.018891509622335434, -0.042716097086668015, 0.03150339424610138, 0.017885349690914154, 0.010300069116055965, 0.007415373809635639, 0.001119169988669455, 0.03000558540225029, 0.013254913501441479, 0.010317797772586346, -0.038159530609846115, 0.002018568804487586, 0.05443934351205826, 0.0350131094455719, -0.0028309214394539595, -0.03034094162285328, -0.002458634553477168, -0.05196539685130119, 0.08079339563846588, 0.02605631947517395, 0.02200884185731411, -0.0010624643182381988, -0.0018097562715411186, 0.06350942701101303, -0.040531355887651443, 0.020099077373743057, -0.02560802735388279, -0.07599270343780518, 0.041047632694244385, 0.029888272285461426, -0.031605031341314316, -0.021313484758138657, 0.005528244189918041, -0.031912125647068024, 0.006632543634623289, 0.012767476961016655, 0.0028874436393380165, 0.04993443191051483, 0.03345254436135292, 0.02847869135439396, -0.0254623088985

In [37]:
test_data['sentiment'] = cls.classify(test_data)['class'].astype(int)
test_data.head()

Columns:
	id	str
	review	str
	vectors	array
	sentiment	int

Rows: 10

Data:
+----------+-------------------------------+-------------------------------+-----------+
|    id    |             review            |            vectors            | sentiment |
+----------+-------------------------------+-------------------------------+-----------+
| 12311_10 | Naturally in a film who's ... | [-0.00566016044468, -0.030... |     1     |
|  8348_2  | This movie is a disaster w... | [-0.00307645485736, 0.0480... |     0     |
|  5828_4  | All in all, this is a movi... | [-0.00124689820223, -0.023... |     1     |
|  7186_2  | Afraid of the Dark left me... | [0.0212037786841, 0.007696... |     0     |
| 12128_7  | A very accurate depiction ... | [0.00801117252558, -0.0185... |     1     |
|  2913_8  | ...as valuable as King Tut... | [0.0336517095566, 0.003712... |     1     |
|  4396_1  | This has to be one of the ... | [-0.0716544240713, 0.02835... |     0     |
|  395_2   | This is one of those movie... | [0.0427797883749, 0.028222... |     1     |
| 10616_1  | The worst movie i've seen ... | [-0.0181394442916, 0.09452... |     0     |
|  9074_9  | Five medical students (Kev... | [0.0220895539969, 0.039015... |     1     |
+----------+-------------------------------+-------------------------------+-----------+
[10 rows x 4 columns]

In [38]:
#saving the prediction to a CSV for submission
test_data[['id', 'sentiment']].save("/Users/marvinbertin/Github/Graphlab/DeepTextpred2.csv", format="csv")
# score 0.859